In [1]:
!pip install trl
# !pip install peft
!pip install sacrebleu
!pip install evaluate
!pip install bert-score
!pip install wandb
!pip install torchmetrics==0.10.3

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
  Using cached trl-0.4.4-py3-none-any.whl (68 kB)
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/sacrebleu-2.3.1+computecanada-py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/portalocker-2.7.0+computecanada-py2.py3-none-any.whl (from sacrebleu)
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/tabulate-0.9.0+computecanada-py3-none-any.whl (from sacrebleu)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)


Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/lxml-4.9.2+computecanada-cp310-cp310-linux_x86_64.whl (from sacrebleu)
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
  Using cached torchmetrics-0.10.3-py3-none-any.whl (529 kB)


In [2]:
# !pip install bitsandbytes

In [3]:
# !pip install -U bitsandbytes

In [4]:
# !pip install -q -U bitsandbytes
# !pip install git+https://github.com/TimDettmers/bitsandbytes

In [5]:
!pip uninstall bitsandbytes
!pip install bitsandbytes

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic/bitsandbytes-0.39.0+computecanada-py3-none-any.whl


In [6]:
!pip uninstall peft --y

In [7]:
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id="EleutherAI/gpt-neo-125m", cache_dir='/scratch/aly2000/lm_extraction/ddp/models/', local_dir_use_symlinks=False)

In [14]:
from dataclasses import dataclass, field
from typing import Optional

import torch
from transformers import GPT2Tokenizer, GPTNeoForCausalLM, AutoModelForCausalLM
from datasets import load_dataset
# from peft import LoraConfig
from tqdm import tqdm
from transformers import AutoTokenizer, HfArgumentParser, pipeline

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler

import torch
import wandb
import time
import os
import statistics
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
from multiprocessing import Pool

from datasets import load_dataset, Dataset

from transformers import AutoTokenizer, pipeline
import argparse
from evaluate import load

from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from torchmetrics.functional import accuracy

In [15]:
from accelerate.tracking import GeneralTracker, on_main_process
from typing import Optional

import wandb


class WandbEd(GeneralTracker):
    name = "wandb"
    requires_logging_directory = False

    @on_main_process
    def __init__(self, project: str, run_name, group: str, entity: str, config: dict):
        self.group = group
        self.run_name = run_name
        self.project = project
        self.config = config
        self.entity = entity
        run = wandb.init(group = self.group, name = self.run_name, entity = self.entity, project = self.project, config=config)

    @property
    def tracker(self):
        return self.run.run

    @on_main_process
    def store_init_configuration(self, values: dict):
        wandb.config = values

    @on_main_process
    def log(self, values: dict, step: Optional[int] = None):
        wandb.log(values)

In [3]:
config_sc = {
    "lm_name": 'facebook/opt-2.7b',
    "alpha_bleu": 0.5,
    "beta_ppl": 0.5,
    "reward_type": 'bert',
    "ref_lm_name": 'facebook/opt-2.7b',
    "cls_model_name": "null",
    "tk_name": "lm_extraction",
    "reward_fn": 'bert',
    "steps": 2,
    "batch_size": 32,
    "forward_batch_size": 8,
    "ppo_epochs": 2,   
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
}


In [4]:
lm_data = pd.read_csv('all_train.csv')
ds = Dataset.from_pandas(lm_data)

In [5]:
import random
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [6]:
mode = 'train'
if mode == 'inference':
    run_name = config_sc['lm_name']+'_'+config_sc['dataset_type']
else:
    run_name = config_sc['lm_name']+'_'+'training'

wandbed_obj = WandbEd(group=config_sc['lm_name'], project='lm_extraction_defence_exps', entity = "thesis_projects", run_name=run_name, config = config_sc)

wandb: Currently logged in as: alykassem (thesis_projects). Use `wandb login --relogin` to force relogin


In [7]:
config = PPOConfig(
    model_name='/scratch/aly2000/lm_extraction/ddp/models/models--facebook--opt-2.7b/snapshots/397f71a473a150c00f0fe3fc4a2f78ff3ccaf82d',
    learning_rate=1.41e-5,
    batch_size=32,
    forward_batch_size=8,
    ppo_epochs = 8,
    optimize_cuda_cache=True,
    remove_unused_columns = False,
    log_with=wandbed_obj,
)

/localscratch/aly2000.5034308.0/jupyter_new/lib/python3.10/site-packages/trl/trainer/ppo_config.py:112: UserWarning: Note that using `forward_batch_size` is deprecated, use `mini_batch_size` instead. By setting it you overwrite `mini_batch_size` which affects both the batch size during forward passes and also the mini batch size for PPO optimization.
  warnings.warn(


In [8]:
from datasets import load_metric
sacrebleu = load_metric('sacrebleu')
bertscore = load("bertscore")

def sacrebleu_fn(label, response):
    score = sacrebleu.compute(predictions=[response], references=[[label]])['score']
    return 100-score 

def calculatePerplexity(sentence):
     """
     exp(loss)
     """
     input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0)
     input_ids = input_ids.to('cuda')
     with torch.no_grad():
         outputs = ppl_model(input_ids, labels=input_ids)
     loss, logits = outputs[:2]
     return torch.exp(loss)

def perplexity_fn(text):
    ppl_lst = []
    for i in text:
        ppl_lst.append(calculatePerplexity(i).unsqueeze(0))
    ppl_tns = torch.cat(ppl_lst)
    return ppl_tns, torch.mean(ppl_tns)


def reward_fn_weighted(bleu_score, ppl_gen):
    bleu_alpha = torch.mul(bleu_score, config_sc['alpha_bleu'])
    ppl_gen_beta = torch.mul(-ppl_gen, config_sc['beta_ppl'])
    score = torch.add(bleu_alpha, ppl_gen_beta)
    return score

def reward_fn_squared(bleu_score, ppl_gen):
    bleu_alpha = torch.mul(bleu_score, config_sc['alpha_bleu'])
    ppl_gen = torch.square(ppl_gen)
    ppl_gen_beta = torch.mul(-ppl_gen, config_sc['beta_ppl'])
    score = torch.add(bleu_alpha, ppl_gen_beta)
    return score

def reward_fn_diff(bleu_score, ppl_gen, ppl_lb):
    bleu_alpha = torch.mul(bleu_score, config_sc['alpha_bleu'])
    diff = torch.sub(ppl_gen, ppl_lb)
    diff_square = torch.square(diff)
    ppl_gen_beta = torch.mul(-diff, config_sc['beta_ppl'])
    score = torch.add(bleu_alpha, ppl_gen_beta)
    return score


def reward_fn_bert_ppl(bert_score, ppl_gen):
    bert_alpha = torch.mul(bert_score, config_sc['alpha_bleu'])
    ppl_gen_beta = torch.mul(ppl_gen, config_sc['beta_ppl'])
    score = torch.add(bert_alpha, ppl_gen_beta)
    return score

def reward_fn_comp(bleu_score, label, response):
    if config_sc['reward_type'] == 'bleu':
        return 'bleu'
    
    if config_sc['reward_type'] == 'weighted':
        reward_pool = Pool()
        score = [reward_pool.apply(reward_fn_weighted, args=(bleu, ppl)) for bleu, ppl in 
                   zip(bleu_score, ppl_gen_tns.cpu().tolist())]
        
    if config_sc['reward_type'] == 'squared':
        reward_pool = Pool()
        score = [reward_pool.apply(reward_fn_weighted, args=(bleu, ppl)) for bleu, ppl in 
                   zip(bleu_score, ppl_gen_tns.cpu().tolist())]      
        
    if config_sc['reward_type'] == 'diff':
        reward_pool = Pool()
        score = [reward_pool.apply(reward_fn_diff, args=(bleu, ppl_gen, ppl_lb)) for bleu, ppl_gen, ppl_lb in 
                   zip(bleu_score, ppl_gen_tns.cpu().tolist(), ppl_lb_tns.cpu().tolist())]
    
    if config_sc['reward_type'] == 'bert':
        score = bertscore.compute(predictions=response, references=label, 
                            model_type="microsoft/deberta-large", device='cuda')['f1']
        score = [-abs(number) for number in score]
        
    if config_sc['reward_type'] == 'bert_ppl':
        reward_pool = Pool()
        bert_scores = bertscore.compute(predictions=response, references=label, 
                            model_type="microsoft/deberta-large", device='cuda')['f1']
        bert_scores = [-abs(number) for number in bert_scores]
        
        score = [reward_pool.apply(reward_fn_bert_ppl, args=(bert_score, ppl)) for bert_score, ppl in 
           zip(bert_scores, ppl_gen_tns.cpu().tolist())]        

    return score

def reward_sacrbleu(response, label):
    pool = Pool()
    result = [pool.apply(sacrebleu_fn, args=(true, pred)) for true, pred in zip(label, response)]
    return result

#Seq impl
def reward_fn(response, label, label_texts, generated_texts):
    
    bleu_score = reward_sacrbleu(response, label)
        
    ppl_lb_tns, ppl_lb_mean = perplexity_fn(label_texts)

    ppl_gen_tns, ppl_gen_mean = perplexity_fn(generated_texts)
    
    score = reward_fn_comp(bleu_score, label, response)
    
    if score == 'bleu':
        score = bleu_score

    return score, bleu_score, ppl_gen_mean, ppl_lb_mean, ppl_lb_tns, ppl_gen_tns

/tmp/ipykernel_233475/1899367765.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  sacrebleu = load_metric('sacrebleu')


In [9]:
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

In [10]:
def tokenize(sample):
    sample["tokens"] = tokenizer.encode(sample["prefix"])
    sample["query"] = tokenizer.decode(sample["tokens"])
    return sample

ds = ds.map(tokenize, batched=False)



gen_kwargs = {
    "min_length":-1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id
}


# tokenizer.pad_token_id = tokenizer.eos_token_id
# gen_kwargs = {
#     "min_length":-1,
#     "top_k": 0.0,
#     "top_p": 1.0,
#     "pad_token_id": tokenizer.eos_token_id
# }

Map:   0%|          | 0/13500 [00:00<?, ? examples/s]

In [11]:
def collater(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [12]:
if mode == 'train':
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        config.model_name,
        device_map="auto")

    model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name, device_map="auto")
elif mode == 'inference':
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        'saved_models/gpt-neo-2.7_bert_8ppo_2eps_nshards',
        device_map="auto")
    
    tokenizer = AutoTokenizer.from_pretrained('saved_models/gpt-neo-2.7_bert_8ppo_2eps_nshards')
    tokenizer.pad_token = tokenizer.eos_token
    
ppl_model = AutoModelForCausalLM.from_pretrained(config.model_name, device_map="auto")    

In [13]:
if mode == 'train':
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)
    # We then build the PPOTrainer, passing the model, the reference model, the tokenizer
    ppo_trainer = PPOTrainer(
        config, model, ref_model=model_ref, tokenizer=tokenizer, dataset=ds, data_collator=collater, optimizer=optimizer
    )
else:
    pass

In [14]:
# text_max_len = max([len(tokenizer.encode(description)) for description in lm_data['suffix'].values])
# print(text_max_len)

In [15]:
def train(total_ppo_epochs=2):
    for epoch, batch in tqdm(zip(range(total_ppo_epochs), iter(ppo_trainer.dataloader))):

        logs, timing = dict(), dict()
        t0 = time.time()
        query_tensors = [torch.tensor(t).long().cuda() for t in batch["tokens"]]

        #### Get response from gpt
        t = time.time()
        response_tensors = []
        for i in range(ppo_trainer.config.batch_size):
            gen_len = 55
            query_tensor_sq = query_tensors[i].unsqueeze(dim=0)
            gen_kwargs["max_new_tokens"] = gen_len
            response = ppo_trainer.generate(query_tensors[i], **gen_kwargs)

            response_tensors.append(response.squeeze()[-gen_len:])

        batch['response'] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
        timing['time/get_response'] = time.time()-t
        #### Compute reward score
        t = time.time()
        respones_batch = batch['response']
        label_batch = batch['suffix']

        label_texts = [q + r for q,r in zip(batch['query'], batch['suffix'])]
        generated_texts = [q + r for q,r in zip(batch['query'], batch['response'])]

        reward_scores, bleu_score, mean_ppl_gen, mean_ppl_label, perplexity_scores_label, perplexity_scores_generated = reward_fn(respones_batch, label_batch, label_texts, generated_texts)
        rewards = torch.tensor(reward_scores, dtype=float).cuda()
        rewards = [torch.tensor(output) for output in rewards]

        timing['time/get_sentiment_preds'] = time.time()-t
        print('finished reward', rewards)
        #### Run PPO step 
        t = time.time()

        model.gradient_checkpointing_enable()
        model.pretrained_model.config.use_cache = False


        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)


        timing['time/optimization'] = time.time()-t

        #### Log everything
        timing['time/epoch'] = time.time()-t0
        rewards = torch.tensor(reward_scores, dtype=float)
        table_rows = [list(r) for r in zip(batch['query'], batch['response'], rewards.cpu().tolist(), batch['suffix'], 
                                           perplexity_scores_label.cpu().tolist(), 
                                           perplexity_scores_generated.cpu().tolist(), bleu_score)]

        logs.update({'game_log': wandb.Table(columns=['query', 'pred', 'reward','label','perplexity_label',
                                                     'perplexity_response', 'bleu_score'], rows=table_rows)})
        logs.update(timing)
    #     logs.update(stats)
        logs['env/reward_mean'] = torch.mean(rewards).cpu().numpy()
        logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
        logs['env/perplexity_gen'] = mean_ppl_gen.cpu().numpy()
        logs['env/perplexity_lab'] = mean_ppl_label.cpu().numpy()
        logs['env/bleu'] = statistics.mean(bleu_score)
        logs['env/reward_dist'] = rewards.cpu().numpy()

        ppo_trainer.accelerator.log(logs)

In [16]:
train(total_ppo_epochs=1)

0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/tmp/ipykernel_233475/1270607966.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rewards = [torch.tensor(output) for output in rewards]
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


finished reward [tensor(-0.5983, device='cuda:0', dtype=torch.float64), tensor(-0.4774, device='cuda:0', dtype=torch.float64), tensor(-0.5873, device='cuda:0', dtype=torch.float64), tensor(-0.6648, device='cuda:0', dtype=torch.float64), tensor(-0.5957, device='cuda:0', dtype=torch.float64), tensor(-0.5511, device='cuda:0', dtype=torch.float64), tensor(-0.3890, device='cuda:0', dtype=torch.float64), tensor(-0.5228, device='cuda:0', dtype=torch.float64), tensor(-0.3864, device='cuda:0', dtype=torch.float64), tensor(-0.7624, device='cuda:0', dtype=torch.float64), tensor(-0.4980, device='cuda:0', dtype=torch.float64), tensor(-0.7642, device='cuda:0', dtype=torch.float64), tensor(-0.7458, device='cuda:0', dtype=torch.float64), tensor(-0.7671, device='cuda:0', dtype=torch.float64), tensor(-0.5704, device='cuda:0', dtype=torch.float64), tensor(-0.7534, device='cuda:0', dtype=torch.float64), tensor(-0.4706, device='cuda:0', dtype=torch.float64), tensor(-0.4382, device='cuda:0', dtype=torch.flo

1it [02:45, 165.46s/it]


In [ ]:
model.save_pretrained('saved_models/opt_2_7b_demem_1eps', max_shard_size='20GB')

In [59]:
cd saved_models/


/scratch/aly2000/lm_extraction/ddp/saved_models


In [ ]:
# idx = 30
# x = ppo_trainer.generate(query_tensors[idx],
#                   **gen_kwargs)
# print(batch['prefix'][idx])
# print('---'*100)
# print(tokenizer.decode(x.squeeze()[-gen_len:]))
# print('---'*100)
# print(batch['suffix'][idx])

In [9]:
def validation_ma(max_len, input_ids):
    labels, preds = [], []
    for i in range(1, max_len):
        label = input_ids[..., i]
        prompt = input_ids[..., :i]
        try:
            pred = model.generate(prompt, max_length=i + 1, pad_token_id=tokenizer.eos_token_id)[:, -1]
        except IndexError:  # if batch == 1
            pred = model.generate(torch.squeeze(
                prompt), max_length=i + 1).squeeze()[-1]

        labels.append(torch.squeeze(label))
        preds.append(torch.squeeze(pred))
    
    preds = torch.stack(preds)
    labels = torch.stack(labels)
    score = accuracy(preds, labels, ignore_index=-100)
    return score

In [10]:
def ngram_of_1D_tensor(X, n):
    grams = [tuple(X[i:i + n].tolist()) for i in range(X.shape[0] - n + 1)]
    return grams

def validation_el(max_len, input_ids):
    
    batch_size = input_ids.shape[0]
    N = [5,10]
    numerator = {n: [0] * batch_size for n in N}
    
    for i in reversed(range(1, max_len)):
        label = input_ids[..., i:max_len]
        prompt = input_ids[..., :i]
        pred = model.generate(prompt, max_length=max_len, pad_token_id=tokenizer.eos_token_id)[..., i:]

        for example_idx in range(batch_size):
            p, l = pred[example_idx], label[example_idx]
            # extraction likelihood
            for n in N:
                p_ngram = ngram_of_1D_tensor(p, n)
                l_ngram = ngram_of_1D_tensor(l, n)
                l_unique = set(l_ngram)
                p_tp = [i for i in p_ngram if i in l_unique]
                try:
                    p_acc = len(p_tp) / len(l_ngram)
                    numerator[n][example_idx] += p_acc
                except ZeroDivisionError:  # n-gram isn't defined
                    pass

    el_score = {n: [0] * batch_size for n in N}
    for n in N:
        for i, _ in enumerate(numerator[n]):
            el_score[n][i] = numerator[n][i] / \
                (max_len - 1 - (n - 1))


    # return individual example results for logging
    ret = {}
    for k in el_score.keys():
        ret[f'el_{k}-gram'] = torch.Tensor(el_score[k])
    return ret

In [17]:
test_data = pd.read_csv('gpt_neo_125m_results.csv')

In [20]:
test_data['ma'] = 0

In [16]:
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125m')
tokenizer.pad_token = tokenizer.eos_token

In [21]:
test_data

,idx,text,pred,true,bleu,ppl_gen,ppl_true,ma
0,0,"gate, Kevin \nMeredith/Corp/Enron@ENRON, Tom M...","Xgate, Mark \nDasovich/NA/Enron@Enron, Mark Fe...","Xgate, Dianne Seib/CAL/ECT@ECT, \nLinda Sietze...",62.468602,3.295946,4.514808,0
1,1,field may be limited in precision depending o...,alling.\n//\n// The type is a struct that is a...,alling. It is generated into a serialization\n...,96.704172,7.584760,15.603815,0
2,2,"do so, subject to the following conditions:\n...","IS"", WITHOUT WARRANTY OF ANY kind,\n* express...","IS"", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR...",70.736053,1.872805,1.526504,0
3,3,value following this digit.\n //\n // Con...,| |\n // V V\n // | |\n // ...,101011\n\n var VLQ_BASE_SHIFT = 5;\n\n // b...,96.957759,2.655847,8.730882,0
4,4,\n # can be virtually everything (everythin...,"""atarist"" to ""atariste""\n # and the proces...","""MiNT"" over ""FreeMiNT""\n # to the lowercas...",96.007783,7.641176,24.066147,0
...,...,...,...,...,...,...,...,...
1495,1495,ft.co/GL_SNS_YT This app allows you to purchas...,golang.org/terms/\n\nThe following is a list o...,gameloft.com/en/conditions-of-use PrivacyPolic...,98.876050,12.530056,14.493207,0
1496,1496,dean [at] fourwalledcubicle [dot] com)\r\n\r\n...,"\n without fee, provided that the following c...","\n without fee, provided that the above copyr...",95.656097,3.831864,4.263714,0
1497,1497,west Realtors shall not be held responsible fo...,change without notice.\n\nThe information con...,adjustments based on your credit score and th...,94.522227,7.099929,21.459770,0
1498,1498,"\n * notice, this list of conditions and the...",derived from this software without specific...,derived from this software without specific...,78.983656,2.160528,1.668214,0


In [27]:
prefix = test_data['text'][0]
inputs = tokenizer([prefix],  return_tensors="pt")
gt = tokenizer(test_data['text'][0] + test_data['true'][0],  return_tensors="pt")
sample = tokenizer(test_data['pred'][0], return_tensors="pt")['input_ids']
gen_len = len(sample[0]) - len(inputs['input_ids'][0]) 
if gen_len>1:
    ma = validation_ma(gen_len, gt['input_ids'].cuda())  

NameError: name 'model' is not defined

In [19]:
test_logs = dict()
test_data = pd.read_csv('test_data.csv')
test_data = test_data.iloc[:50]
# test_data = test_data.sample(50)
# test_data.reset_index(inplace=True, drop=True)
# test_data.reset_index(inplace=True, drop=True)
test_scores = pd.DataFrame(columns=['idx','text','pred','true', 'bleu', 'ppl_gen', 'ppl_true', 'ma'])
    
for i in tqdm(range(len(test_data))):
    prefix = test_data['prefix'][i]
    inputs = tokenizer([prefix],  return_tensors="pt")
    sample  = model.generate(**inputs.to('cuda'), max_length=103, pad_token_id=tokenizer.eos_token_id)

    suffix_gen = tokenizer.decode(sample[0])
    sample_text = suffix_gen[len(prefix):]
    suffix = test_data['suffix'][i]
    
    ppl_true = calculatePerplexity(prefix + suffix)
    ppl_gen = calculatePerplexity(prefix + sample_text)
    bleu = sacrebleu_fn(suffix, sample_text)
    
    gt = tokenizer(test_data['prefix'][i] + test_data['suffix'][i],  return_tensors="pt")
    gen_len = len(sample[0]) - len(inputs['input_ids'][0]) 
    if gen_len>1:
        ma = validation_ma(gen_len, gt['input_ids'].cuda())    
    #     el = validation_el(gen_len, gt['input_ids'].cuda())

        test_scores.loc[i ,'idx'] = i
        test_scores.loc[i, 'text'] = prefix
        test_scores.loc[i, 'pred'] = sample_text
        test_scores.loc[i, 'true'] = suffix
        test_scores.loc[i, 'bleu'] = bleu
        test_scores.loc[i, 'ppl_gen'] = ppl_gen.cpu().tolist()
        test_scores.loc[i, 'ppl_true'] = ppl_true.cpu().tolist()
        test_scores.loc[i, 'ma'] = ma.cpu().tolist()
    else:
        pass
#     test_scores.loc[i, 'el_5_gram'] = el['el_5-gram'].cpu().tolist()
#     test_scores.loc[i, 'el_10_gram'] = el['el_10-gram'].cpu().tolist()

  0%|                                                                                                            | 0/50 [00:00<?, ?it/s]/localscratch/aly2000.5034308.0/jupyter_new/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:48<00:00,  5.76s/it]


In [20]:
test_scores['bleu'].mean()

87.11677999599152

In [28]:
# 73.89272666106629

,idx,text,pred,true,bleu,ppl_gen,ppl_true,ma
0,0,"gate, Kevin \nMeredith/Corp/Enron@ENRON, Tom M...","nronXgate, Tana\nJones/HOU/ECT@ECT, Mark Taylo...","Xgate, Dianne Seib/CAL/ECT@ECT, \nLinda Sietze...",66.658828,5.344698,7.289,0.588235
1,1,field may be limited in precision depending o...,arshalling.\n int32 nanos = 2;\n}\n\n// Times...,alling. It is generated into a serialization\n...,96.688482,12.023802,43.264538,0.333333
2,2,"do so, subject to the following conditions:\n...","""AS IS"", WITHOUT WARRANTY OF ANY KIND, EXPRES...","IS"", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR...",11.954528,3.167221,2.779714,0.607843
3,3,value following this digit.\n //\n // Con...,// | |\n // | |\n // | |\n /...,101011\n\n var VLQ_BASE_SHIFT = 5;\n\n // b...,96.850738,4.19387,13.524801,0.568627
4,4,\n # can be virtually everything (everythin...,"from ""0"" to ""1"" (the\n # ""0"" is the system...","""MiNT"" over ""FreeMiNT""\n # to the lowercas...",90.605346,13.746076,31.504759,0.313726
5,5,"Linklayer, nil\n}\n\n// SockaddrNetlink implem...",uint16\n\tSockaddr uint16\n\tSockaddrNetlin...,16\n\tPid uint32\n\tGroups uint32\n\traw ...,98.743246,10.634298,25.02775,0.392157
6,6,"h3>\n<table class=""overviewSummary"" border=""0""...",pan>Nested Class Summary</span></caption>\n<ta...,"Nested Classes</span><span class=""tabEnd"">&nbs...",77.281921,6.777209,13.078873,0.313726
7,7,M110.69 86.593c-2.961 2.678-5.868 6.012-8.437 ...,9 18.957-9.838\n\nM110.69 86.593c-2.961 2.678-...,.136 2.79 4.723 5.332 7.065 7.742 7.146l.076.0...,97.44595,10.575316,42.101574,0.352941
8,8,or\n// (at your option) any later version.\n/...,LITY or FITNESS FOR A PARTICULAR PURPOSE. See ...,or FITNESS FOR A PARTICULAR PURPOSE. See the...,19.552061,4.848588,4.278004,0.372549
9,9,state += 'Text'\n\n if (data.resetText =...,"it\n // if the form is submitted, then the ...",setTimeout($.proxy(function () {\n $el...,99.167822,14.094247,25.615978,0.27451


In [ ]:
test_scores['bleu'].mean()

In [21]:
test_scores_demem = test_scores.copy()

In [22]:
test_scores_demem['bleu'].mean()

93.60600447511368

In [25]:
test_scores_demem['ppl_true'].mean()

23.055712027549745

In [26]:
test_scores_demem['ppl_gen'].mean()

10.007952604293823

In [ ]:
# org_1.3b: 71.78519546186536

In [50]:
w = pd.read_csv('dfs_results/gpt_neo_2.7_reg.csv')

In [53]:
w.iloc[:376]['bleu'].mean()

63.88908980638253

In [ ]:
# 4 eps: 91.69438161977482
# org: 93.88984732218921

In [34]:
##old
test_scores['bleu'].mean()

32.84758026782263

In [35]:
(0.4089822744+ 0.568600815059188+0.5603788477+0.78+0.5951178451+0.2773037543+0.7072905332+0.2405360134+0.583)/9

0.5245788981287987

In [36]:
(0.4271+ 0.6224+0.577+0.79+0.6106902357+0.2747440273+0.7219804135+0.2472361809+0.565)/9

0.5373500952666667

In [ ]:
wandb.log({"test_samples": test_scores})
    
test_logs['test/ppl_gen'] = test_scores['ppl_gen'].mean()
test_logs['test/ppl_true'] = test_scores['ppl_true'].mean()
test_logs['test/bleu'] = test_scores['bleu'].mean()
wandb.log(test_logs)